In [173]:
import pandas as pd
import numpy as np
import os
import re

In [174]:
# Function to return the sum of elements in an array
def findSum(arr):
    sum = 0
    for element in arr:
        sum += element
    return sum


In [175]:
def calculate_event_time(event_df, sub_id, trial_id, condition):
    event_list = []
    total_elapsed_time = 0
    updated_event_df = pd.DataFrame()
    
    # iterate through all the rows in the dataframe and store them as dicts in a list
    for idx, row in event_df.iterrows():
        row = row.to_dict()
        event_list.append(row)

    # Find all the target find times and store them in an array for later
    find_times = []
    for event in event_list:
        find_times.append(event['TargetFinding Time'])

    for idx, events in enumerate(event_list):
        target_area = events['Target Area']
        target_sign = events[' Target sign']
        target_finding_result = events[' TargetFinding result']
        target_finding_time = events['TargetFinding Time']

        # print(events)

        default_elapsed_time = 5 if idx == 0 else (idx + 1) * 5
        elapsed_find_times = find_times[:idx]
        sum_find_times = findSum(elapsed_find_times)
        total_elapsed_time = sum_find_times + default_elapsed_time
        
        this_df = pd.DataFrame(
                {'target_area': target_area, 
                    'target_sign': target_sign,
                    'target_finding_time': target_finding_time, 
                    'target_finding_result': target_finding_result,
                    'sub_id': sub_id,
                    'trial_id': trial_id,
                    'condition': condition, 
                    'seconds_after_trigger': total_elapsed_time}, index=[0])
        
        updated_event_df = pd.concat([updated_event_df, this_df], ignore_index=False)
    return updated_event_df



In [178]:
ignored_dirs=['.ipynb_checkpoints']
all_events = pd.DataFrame()
behavior_files = os.listdir("../behavior_data")
behavior_files = list(filter(lambda file: file not in ignored_dirs, behavior_files))
for file in behavior_files:
    event_df = pd.read_csv(f"../behavior_data/{file}")
    
# find meta data from the CSV file name
    # remove ".csv"
    file = file.split('.')[0]
    # split by uppercase
    name_list = re.findall('[A-Z][^A-Z]*', file)
    # isolate user ID
    sub_id = re.split("(\d)", name_list[0])[1]
    
    #isolate the trial_id
    trial_id = re.split("(\d)", name_list[-1])[1]

    # Figure out the condition
    condition = re.split(r"trial", name_list[-1])[0]
    
    # Remove the events that didn't occur
    mask = event_df['TargetFinding Time'] != -1
    event_df = event_df[mask]
    
    # Figure out the time that each event occured relative to the trigger
    df = calculate_event_time(event_df, sub_id, trial_id, condition)
    all_events = pd.concat([df, all_events], ignore_index=True)

# remove the 0 find_time events
final_mask = all_events['target_finding_time'] != 0
all_events = all_events[final_mask]    
all_events.to_csv('attention_shift_events.csv')

